# UserVariance

In [ ]:
medium = ""

In [ ]:
const name = "$medium/all/UserVariance"
const content = "explicit";

In [ ]:
using DataFrames

import NBInclude: @nbinclude
import Statistics: var
@nbinclude("../Alpha.ipynb");

In [ ]:
function get_dataframe(split, content)
    df = get_split(split, "all", content, medium)
    DataFrame(user = df.user, item = df.item, rating = df.rating)
end;

In [ ]:
function make_prediction(users, items, u)
    r = zeros(Float32, length(users))
    Threads.@threads for i = 1:length(r)
        if users[i] in keys(u)
            r[i] = u[users[i]]
        end
    end
    r
end;

In [ ]:
const training = get_dataframe("training", content)
u = combine(groupby(training, :user), :rating => var => :rating)
u.rating[isnan.(u.rating)] .= 0
u = Dict(Pair.(u.user, u.rating))
model(users, items) = make_prediction(users, items, u)
write_params(Dict("u" => u), name)
write_alpha(model, medium, name; log = false);